# AI-Дублёр Голоса  
**Проект на Hugging Face:** распознавание русской речи и озвучка текста.  
Модели:  
- `openai/whisper-small` — Speech-to-Text (распознаёт речь на русском)  
- `snakers4/silero-models` — Text-to-Speech (озвучивает текст по-русски)


Установка зависимостей

In [1]:
!pip install -q transformers torch soundfile


Импорт библиотек

In [2]:
from transformers import pipeline
import torch
import soundfile as sf
from IPython.display import Audio


## 1. Загружаем аудио
Здесь можно:
- Использовать готовое демо-аудио, или  
- Загрузить свой `.wav` / `.mp3` файл (русская речь, 5–10 секунд).  


Пример загрузки файла

In [3]:
from google.colab import files

print("Загрузите аудио-файл (.wav или .mp3):")
uploaded = files.upload()

audio_path = list(uploaded.keys())[0]
print(f"Загружен файл: {audio_path}")


Загрузите аудио-файл (.wav или .mp3):


Saving тест .mp3 to тест  (1).mp3
Загружен файл: тест  (1).mp3


## 2. Распознаём речь при помощи Whisper
Модель `openai/whisper-small` поддерживает русский язык.  


Код распознавания речи

In [4]:
asr = pipeline("automatic-speech-recognition", model="openai/whisper-small")

result = asr(audio_path, generate_kwargs={"language": "ru"})
recognized_text = result["text"]

print("Распознанный текст:")
print(recognized_text)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

preprocessor_config.json: 0.00B [00:00, ?B/s]

Device set to use cpu
`return_token_timestamps` is deprecated for WhisperFeatureExtractor and will be removed in Transformers v5. Use `return_attention_mask` instead, as the number of frames can be inferred from it.


Распознанный текст:
 Сегодня 16 октября 2025 года мы пытаемся разобраться с тем, что нам нужно сделать.


## 3. Озвучиваем текст с помощью Silero TTS
Модель `snakers4/silero-models` создаёт русскую речь.  
Можно использовать распознанный текст или написать свой.


In [5]:
torch.hub._validate_not_a_forked_repo=lambda a,b,c: True
model, example_text = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                     model='silero_tts',
                                     language='ru',
                                     speaker='v3_1_ru')

text_to_say = recognized_text  # можно заменить на любой свой текст
audio = model.apply_tts(text=text_to_say, speaker='baya', sample_rate=48000)

sf.write('tts_output.wav', audio.numpy(), 48000)
print("Синтез завершён: tts_output.wav")
Audio('tts_output.wav')


/usr/local/lib/python3.12/dist-packages/torch/hub.py:335: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(


Downloading: "https://github.com/snakers4/silero-models/zipball/master" to /root/.cache/torch/hub/master.zip


100%|██████████| 59.0M/59.0M [00:03<00:00, 18.7MB/s]


✅ Синтез завершён: tts_output.wav


## 4. Отредактируй текст и озвучь заново



In [6]:
custom_text = input("Введите текст для озвучивания: ")

if custom_text.strip():
    custom_audio = model.apply_tts(text=custom_text, speaker='baya', sample_rate=48000)
    sf.write('tts_custom.wav', custom_audio.numpy(), 48000)
    print("Новое аудио готово: tts_custom.wav")
    Audio('tts_custom.wav')
else:
    print("Пустая строка, ничего не озвучено.")


Введите текст для озвучивания: Проверка моя любимая
Новое аудио готово: tts_custom.wav
